In [ ]:
#@title Kaggle API

from IPython.display import clear_output

username = str(input("username: "))
key = str(input("key: "))

clear_output()

f = open("kaggle.json", "w")
f.write('{"username":"'+username+'","key":"'+key+'"}')
f.close()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d hernan4444/anime-recommendation-database-2020

import os, re
from zipfile import ZipFile

with ZipFile('/content/anime-recommendation-database-2020.zip', 'r') as zipObj:
   zipObj.extractall()

def purge(dir, pattern):
    for f in os.listdir(dir):
        if pattern in f:
            os.remove(os.path.join(dir, f))

purge("/content/", ".zip")
purge("/content/", ".json")

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Data Preprocessing

In [5]:
ratings = pd.read_csv("/content/animelist.csv")

In [6]:
ratings.head(5)

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,4898,0,1,1
4,0,21,10,1,0


In [7]:
ratings["rating"].value_counts()

0     46827035
8     15422150
7     14244633
9     10235934
6      7543377
10     7144392
5      4029645
4      1845854
3       905700
2       545339
1       480688
Name: rating, dtype: int64

In the dataset, if a person hasn't left a rating to an anime, it was marked as a zero. Let's remove the zeros, as they will just introduce noise later on. Might as well remove the `watching_status` and `watched_episodes`. They are usefull pieces of data, but we won't use them in this tutorial.

In [8]:
ratings = ratings[ratings["rating"] > 0]
ratings = ratings.drop(columns = ["watching_status", "watched_episodes"])

In [9]:
num_users = ratings["user_id"].nunique()
num_anime = ratings["anime_id"].nunique()

print(f"There are a total of {num_users} users found.")
print(f"There are a total of {num_anime} anime found.")

print("Max user id:", ratings["user_id"].max())
print("Max anime id:", ratings["anime_id"].max())

There are a total of 313670 users found.
There are a total of 17172 anime found.
Max user id: 353404
Max anime id: 48456


If we take a look at the anime and user ids, we can notice that some values there are missing. Let's make a two helper functions, for easy conversion between dataset ids and embedding ids.

In [10]:
embId2user = sorted(ratings["user_id"].unique())
embId2anime = sorted(ratings["anime_id"].unique())

In [11]:
user2embId = {v: k for k, v in enumerate(embId2user)}
anime2embId = {v: k for k, v in enumerate(embId2anime)}

Now, let's make the dataset. It will look like a tuples of `(user_id, anime_embId, rating)`. We'll make 3 sets: `train`, `test` and `val`.

In [12]:
np.random.seed(seed = 42)

def makeDataSet(df, split=0.95):
    n = df.to_numpy()
    
    n = np.random.permutation(n)

    x = n[:, :2]
    y = n[:, 2]

    for i in range(x.shape[0]):
        x[i][0] = user2embId[x[i][0]]
        x[i][1] = anime2embId[x[i][1]]

    s1 = int(split * n.shape[0])
    s2 = s1 + int((1 - split) * n.shape[0] / 10)
    
    return (x[:s1], y[:s1], x[s1:s2], y[s1:s2], x[s2:], y[s2:])

In [13]:
x_train, y_train, x_test, y_test, x_val, y_val = makeDataSet(ratings)

# Models

In [14]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Matrix Factorization

In [40]:
class MatrixFactorizationModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim):
        super(MatrixFactorizationModel, self).__init__()
        
        self.embedding_dim = embedding_dim
        
        self.user_embeddings = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embeddings = tf.keras.layers.Embedding(num_items, embedding_dim)

        self.user_biases = tf.keras.layers.Embedding(num_users, 1)
        self.item_biases = tf.keras.layers.Embedding(num_items, 1)

        self.bias = tf.Variable(tf.zeros([1]))

        self.dropout = tf.keras.layers.Dropout(.5)

    def call(self, inputs, training = False):
        user_ids = inputs[:, 0]
        item_ids = inputs[:, 1]

        user_embedding = self.user_embeddings(user_ids) + self.user_biases(user_ids)
        item_embedding = self.item_embeddings(item_ids) + self.item_biases(item_ids)

        if training:
            user_embedding = self.dropout(user_embedding, training = training)
            item_embedding = self.dropout(item_embedding, training = training)

        user_embedding = tf.reshape(user_embedding, [-1, self.embedding_dim])
        item_embedding = tf.reshape(item_embedding, [-1, self.embedding_dim])

        dot = tf.keras.layers.Dot(axes=1)([user_embedding, item_embedding]) + self.bias

        return dot
    def plot_model(self, 
                   to_file = 'model.png', dpi = 96, 
                   show_shapes = True, 
                   show_layer_names = True, 
                   expand_nested = True):
        x = tf.keras.layers.Input(shape=(2, 1))

        return tf.keras.utils.plot_model(
            tf.keras.Model(inputs=[x], outputs = self.call(x)),
            to_file = to_file, 
            dpi = dpi,
            show_shapes = show_shapes, 
            show_layer_names = show_layer_names,
            expand_nested = expand_nested,
            # rankdir = 'LR'  
        )

In [43]:
mf_model = MatrixFactorizationModel(num_users = num_users, 
                                    num_items = num_anime, 
                                    embedding_dim = 64)

mf_model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.MeanSquaredError(),
    metrics = [
        tf.keras.metrics.RootMeanSquaredError("RMSE")
    ],
    run_eagerly = True
)

In [44]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)

history = mf_model.fit(x = x_train, y = y_train, 
                    batch_size = 64, 
                    epochs = 100, 
                    steps_per_epoch = 1000,
                    callbacks = [callback],
                    validation_data = (x_test, y_test),
                    validation_steps = 10,
                    validation_batch_size = 64
                    )

Epoch 1/100
1000/1000 [==============================] - 40s 36ms/step - loss: 51.3878 - RMSE: 7.1685 - val_loss: 43.5188 - val_RMSE: 6.5969
Epoch 2/100
1000/1000 [==============================] - 35s 35ms/step - loss: 39.2640 - RMSE: 6.2661 - val_loss: 32.6083 - val_RMSE: 5.7104
Epoch 3/100
1000/1000 [==============================] - 35s 35ms/step - loss: 29.2908 - RMSE: 5.4121 - val_loss: 23.8823 - val_RMSE: 4.8870
Epoch 4/100
1000/1000 [==============================] - 35s 35ms/step - loss: 21.2222 - RMSE: 4.6068 - val_loss: 16.9432 - val_RMSE: 4.1162
Epoch 5/100
1000/1000 [==============================] - 35s 35ms/step - loss: 14.9957 - RMSE: 3.8724 - val_loss: 11.6709 - val_RMSE: 3.4163
Epoch 6/100
1000/1000 [==============================] - 36s 36ms/step - loss: 10.2482 - RMSE: 3.2013 - val_loss: 7.8572 - val_RMSE: 2.8031
Epoch 7/100
1000/1000 [==============================] - 36s 36ms/step - loss: 6.8801 - RMSE: 2.6230 - val_loss: 5.3622 - val_RMSE: 2.3156
Epoch 8/100
1000

In [46]:
# mf_model.save_weights(
#    "/content/drive/Shareddrives/ML/Anime RecSys/MatrixFactorizationModel/model", overwrite=True
# )

In [ ]:
mf_model.load_weights(
    "/content/drive/Shareddrives/ML/Anime RecSys/MatrixFactorizationModel/model"
)

## Neural Network

In [48]:
class NeuralNetworkModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim):
        super(NeuralNetworkModel, self).__init__()
        
        self.embedding_dim = embedding_dim
        
        self.user_embeddings = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embeddings = tf.keras.layers.Embedding(num_items, embedding_dim)

        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='relu')

        self.concat = tf.keras.layers.Concatenate()
        self.dropout = tf.keras.layers.Dropout(.5)

    def call(self, inputs, training = False):
        user_ids = inputs[:, 0]
        item_ids = inputs[:, 1]

        user_embedding = self.user_embeddings(user_ids)
        item_embedding = self.item_embeddings(item_ids)

        if training:
            user_embedding = self.dropout(user_embedding, training = training)
            item_embedding = self.dropout(item_embedding, training = training)

        user_embedding = tf.reshape(user_embedding, [-1, self.embedding_dim])
        item_embedding = tf.reshape(item_embedding, [-1, self.embedding_dim])

        x = self.concat([user_embedding, item_embedding])
        x = self.dense1(x)
        x = self.dense2(x)

        return x
    def plot_model(self, 
                   to_file = 'model.png', dpi = 96, 
                   show_shapes = True, 
                   show_layer_names = True, 
                   expand_nested = True):
        x = tf.keras.layers.Input(shape=(2, 1))

        return tf.keras.utils.plot_model(
            tf.keras.Model(inputs=[x], outputs = self.call(x)),
            to_file = to_file, 
            dpi = dpi,
            show_shapes = show_shapes, 
            show_layer_names = show_layer_names,
            expand_nested = expand_nested,
            # rankdir = 'LR'  
        )

In [49]:
nn_model = NeuralNetworkModel(num_users = num_users, 
                              num_items = num_anime, 
                              embedding_dim = 64)

nn_model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.MeanSquaredError(),
    metrics = [
        tf.keras.metrics.RootMeanSquaredError("RMSE")
    ],
    run_eagerly = True
)

In [50]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)

history = nn_model.fit(x = x_train, y = y_train, 
                       batch_size = 64, 
                       epochs = 100, 
                       steps_per_epoch = 1000,
                       callbacks = [callback],
                       validation_data = (x_test, y_test),
                       validation_steps = 10,
                       validation_batch_size = 64
                    )

Epoch 1/100
1000/1000 [==============================] - 31s 29ms/step - loss: 10.0091 - RMSE: 3.1637 - val_loss: 2.7234 - val_RMSE: 1.6503
Epoch 2/100
1000/1000 [==============================] - 29s 29ms/step - loss: 2.6697 - RMSE: 1.6339 - val_loss: 2.5527 - val_RMSE: 1.5977
Epoch 3/100
1000/1000 [==============================] - 29s 29ms/step - loss: 2.5479 - RMSE: 1.5962 - val_loss: 2.4375 - val_RMSE: 1.5612
Epoch 4/100
1000/1000 [==============================] - 28s 28ms/step - loss: 2.4327 - RMSE: 1.5597 - val_loss: 2.3629 - val_RMSE: 1.5372
Epoch 5/100
1000/1000 [==============================] - 29s 29ms/step - loss: 2.4151 - RMSE: 1.5541 - val_loss: 2.3518 - val_RMSE: 1.5336
Epoch 6/100
1000/1000 [==============================] - 29s 29ms/step - loss: 2.3623 - RMSE: 1.5370 - val_loss: 2.3340 - val_RMSE: 1.5277
Epoch 7/100
1000/1000 [==============================] - 28s 28ms/step - loss: 2.3262 - RMSE: 1.5252 - val_loss: 2.2111 - val_RMSE: 1.4870
Epoch 8/100
1000/1000 [===

In [52]:
# nn_model.save_weights(
#    "/content/drive/Shareddrives/ML/Anime RecSys/NeuralNetworkModel/model", overwrite=True
# )

In [ ]:
nn_model.load_weights(
    "/content/drive/Shareddrives/ML/Anime RecSys/NeuralNetworkModel/model"
)

## Comparing the models

In [53]:
mf_model.evaluate(x = x_val, y = y_val)

87747/87747 [==============================] - 959s 11ms/step - loss: 2.8429 - RMSE: 1.6861


[2.842893600463867, 1.68608820438385]

In [54]:
nn_model.evaluate(x = x_val, y = y_val)

87747/87747 [==============================] - 858s 10ms/step - loss: 1.9897 - RMSE: 1.4105


[1.9896513223648071, 1.4105499982833862]